# Eigenvector Centrality

In [31]:
# Imports
from __future__ import division
from collections import *
from functools import *

import math,re,random
import numpy as np
import matplotlib.pyplot as plt

In [32]:
# dot, get_row, get_column, make_matrix, magnitude, scalar_multiply, shape, distance
def dot(v,w):
    return sum([v_i*w_i for v_i,w_i in zip(v,w)])
def get_row(A,i):
    return A[i]
def get_column(A,j):
    return [A_i[j] for A_i in A]
def make_matrix(num_rows,num_columns,entry_fn):
    return [[entry_fn(i,j) for j in range(num_columns)]
           for i in range(num_rows)]
def magnitude(v):
    return math.sqrt(dot(v,v))
def scalar_multiply(c,v):
    return [c*v_i for v_i in v]
def shape(A):
    num_rows = len(A)
    num_columns = len(A[0]) if A else 0
    return num_rows,num_columns
def distance(v,w):
    return math.sqrt(sum([(v_i - w_i)**2
                           for v_i,w_i in zip(v,w)]))

In [33]:
def matrix_product_entry(A,B,i,j):
    return dot(get_row(A,i),get_column(B,j))
def matrix_multiply(A,B):
    n1,k1 = shape(A)
    n2,k2 = shape(B)
    if k1 != n2:
        raise ArithmeticError("Incompatible Shape")
    return make_matrix(n1,k2,partial(matrix_product_entry,A,B))

In [34]:
def vector_as_matrix(v):
    return [[v_i] for v_i in v]
def vector_from_matrix(v_as_matrix):
    return [row[0] for row in v_as_matrix]

In [35]:
def matrix_operator(A, v):
    v_as_matrix = vector_as_matrix(v)
    product = matrix_multiply(A,v_as_matrix)
    return vector_from_matrix(product)

Suppose there is a matrix A ,of size ``n x n`` and another vector v , of size ``n x 1``. When we multiply A x v , the result will be of size ``n x 1``, some times the result can be a scalar multiply of v . That is, that the result is a vector that points in the same direction as v . When this happens (and when, in addition, v is not a vector of all zeroes), we call v an eigenvector of A. And we call the multiplier an eigenvalue.

In [36]:
# One way is to get a random v and adjust
def find_eigenvector(A,tolerance = 0.00001):
    guess = [random.random() for _ in A]
    while True:
        result = matrix_operator(A,guess)
        length = magnitude(result)
        next_guess = scalar_multiply(1/length,result)
        
        if distance(guess,next_guess) < tolerance:
            return next_guess,length
        guess = next_guess

In [37]:
find_eigenvector([[2,5],[6,9]])

([0.4472139215280274, 0.8944270279858068], 11.99999562576164)

In [38]:
users = [
    { "id": 0, "name": "Hero" },
    { "id": 1, "name": "Dunn" },
    { "id": 2, "name": "Sue" },
    { "id": 3, "name": "Chi" },
    { "id": 4, "name": "Thor" },
    { "id": 5, "name": "Clive" },
    { "id": 6, "name": "Hicks" },
    { "id": 7, "name": "Devin" },
    { "id": 8, "name": "Kate" },
    { "id": 9, "name": "Klein" }
]

friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
               (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

In [39]:
def entry_fn(i,j):
    return 1 if (i,j) in friendships or (j,i) in friendships else 0
n = len(users)
adjuscend_matrix = make_matrix(n,n,entry_fn)

In [40]:
print(adjuscend_matrix)

[[0, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1, 1, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]


In [41]:
eigenvector_centralities, _ = find_eigenvector(adjuscend_matrix)

In [42]:
print(eigenvector_centralities)

[0.3857811883270553, 0.5147900583468422, 0.5147900583468422, 0.4733141199102854, 0.23360707059776406, 0.15014813417789946, 0.0835506726957974, 0.0835506726957974, 0.07284220463736601, 0.027292271037443663]


In [43]:
endorsements = [(0, 1), (1, 0), (0, 2), (2, 0), (1, 2),
(2, 1), (1, 3), (2, 3), (3, 4), (5, 4),
(5, 6), (7, 5), (6, 8), (8, 7), (8, 9)]

In [44]:
for user in users:
    user["endorses"] = []
    user["endorsed_by"] = []
for source_id,target_id in endorsements:
    users[source_id]["endorses"].append(users[target_id])
    users[target_id]["endorsed_by"].append(users[source_id])
endorsements_by_id = [(user["id"], len(user["endorsed_by"]))
                        for user in users]
sorted(endorsements_by_id,
    key=lambda pair: pair[1],
    reverse=True)

[(0, 2),
 (1, 2),
 (2, 2),
 (3, 2),
 (4, 2),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [45]:
def page_rank(users, damping = 0.85, num_iters = 100):

    # initially distribute PageRank evenly
    num_users = len(users)
    pr = { user["id"] : 1 / num_users for user in users }

    # this is the small fraction of PageRank
    # that each node gets each iteration
    base_pr = (1 - damping) / num_users

    for ni in range(num_iters):
        next_pr = { user["id"] : base_pr for user in users }
        print(ni," => ",pr)
        for user in users:
            # distribute PageRank to outgoing links
            links_pr = pr[user["id"]] * damping
            for endorsee in user["endorses"]:
                next_pr[endorsee["id"]] += links_pr / len(user["endorses"])

        pr = next_pr

    return pr

In [46]:
page_rank(users)

0  =>  {0: 0.1, 1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0.1, 6: 0.1, 7: 0.1, 8: 0.1, 9: 0.1}
1  =>  {0: 0.07166666666666667, 1: 0.08583333333333334, 2: 0.08583333333333334, 3: 0.07166666666666667, 4: 0.14250000000000002, 5: 0.1, 6: 0.05750000000000001, 7: 0.05750000000000001, 8: 0.1, 9: 0.05750000000000001}
2  =>  {0: 0.0636388888888889, 1: 0.06977777777777779, 2: 0.06977777777777779, 3: 0.0636388888888889, 4: 0.11841666666666667, 5: 0.06387500000000002, 6: 0.05750000000000001, 7: 0.05750000000000001, 8: 0.06387500000000002, 9: 0.05750000000000001}
3  =>  {0: 0.05454074074074075, 1: 0.061816898148148156, 2: 0.061816898148148156, 3: 0.05454074074074075, 4: 0.09623993055555556, 5: 0.06387500000000002, 6: 0.04214687500000001, 7: 0.04214687500000001, 8: 0.06387500000000002, 9: 0.04214687500000001}
4  =>  {0: 0.05002957561728395, 1: 0.055694602623456796, 2: 0.055694602623456796, 3: 0.05002957561728395, 4: 0.08850650462962964, 5: 0.050824843750000015, 6: 0.04214687500000001, 7: 0.0421468750000000

{0: 0.0404553415061296,
 1: 0.044921190893169885,
 2: 0.044921190893169885,
 3: 0.0404553415061296,
 4: 0.06785083675770529,
 5: 0.04344422700587085,
 6: 0.03346379647749512,
 7: 0.03346379647749512,
 8: 0.04344422700587085,
 9: 0.03346379647749512}